# Autoencoder CIFAR-10 Training and Experimentation

This notebook demonstrates training Autoencoder model on the CIFAR-10 dataset using a modular training pipeline implemented in PyTorch Lightning.


In [ ]:
!git clone !git clone https://github.com/Reennon/gen-ai-cv-lab-1.git
%cd gen-ai-cv-lab-1
!pip install -r requirements.txt

In [2]:
%cd ..

/Users/rkovalch/Documents/UCU/gen-ai/gen-ai-cv-lab-1


/Users/rkovalch/Library/Caches/pypoetry/virtualenvs/gen-ai-cv-lab-1-MbMkjl16-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
import os
import dotenv
import wandb
import torch

from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from src.visualization.base_visualizer import BaseVisualizer
from src.training.trainer import train_model
from src.models.autoencoder import Autoencoder


In [3]:
dotenv.load_dotenv()

True

In [4]:
parameters = OmegaConf.load("./params/autoencoder.yml")
wandb.login(key=os.environ["WANDB_KEY"])

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rkovalch (rkovalchuk). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/rkovalch/.netrc


True

In [5]:
wandb_project_name = "cifar-10-autoencoder"
device = "mps" #"cuda:0"

In [6]:
# Define data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

# Load datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [7]:
dataiter = iter(train_loader)
images, labels = next(dataiter)
print(torch.min(images), torch.max(images))

tensor(-1.) tensor(1.)


In [8]:
images.shape

torch.Size([64, 3, 32, 32])

In [9]:
hparams = parameters.hyperparameters

In [10]:
# Edit hparams dict here as experiemnt, wandb will log the difference
hparams["lr"] = 3e-3

dict(hparams)

{'lr': 0.003, 'epochs': 10, 'latent_dim': 64}

In [11]:
# Train the Autoencoder
train_model(Autoencoder, parameters, train_loader, val_loader, wandb_project_name)


TypeError: Trainer.__init__() got an unexpected keyword argument 'gpus'

In [ ]:
# Visualize metrics from wandb
from IPython.display import display
wandb_url = wandb.run.url
display(f"Wandb Dashboard: {wandb_url}")


In [ ]:
# Load the model
model = Autoencoder(hparams)
model.load_from_checkpoint('path/to/best_checkpoint.ckpt')
model.eval()

# Visualize original and reconstructed images
visualizer = BaseVisualizer(model, val_loader)
visualizer.visualize_reconstructions()